<img src="logo.png">

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
datos = pd.read_csv("datos_procesamiento.csv")
datos.head()

In [ ]:
datos.dtypes

In [ ]:
datos.shape

# Variables Numéricas

**Imputación de datos**

In [ ]:
from sklearn import preprocessing

In [ ]:
datos.dtypes

In [ ]:
var_numericas_df = datos.select_dtypes([int, float])
var_numericas_df["col_outliers"] = datos["col_outliers"]
var_numericas_df.columns

In [ ]:
var_numericas_df[var_numericas_df.isnull().any(axis=1)].shape

In [ ]:
var_numericas_df[var_numericas_df.isnull().any(axis=1)].head()

In [ ]:
from sklearn.impute import SimpleImputer
imputador = SimpleImputer(missing_values=np.nan, copy=False, strategy="mean")

In [ ]:
var_numericas_imputadas = imputador.fit_transform(var_numericas_df)

In [ ]:
var_numericas_imputadas

In [ ]:
var_numericas_imputadas.shape

In [ ]:
var_numericas_imputadas_df = pd.DataFrame(var_numericas_imputadas,
                                                   index=var_numericas_df.index,
                                                   columns=var_numericas_df.columns)

var_numericas_imputadas_df.head(10)

In [ ]:
var_numericas_imputadas_df[var_numericas_imputadas_df.isnull().any(axis=1)].shape

**Estandarización**

El proceso de Estandarización es un proceso requerido por una gran cantidad de modelos en Scikit-learn. El objetivo es obtener una variable con media 0 y desviación estándar 1.

In [ ]:
var_numericas_df.columns

In [ ]:
var_numericas_df.mean()

In [ ]:
var_numericas_df.std()

Para ello el transformador mas sencillo en sklearn es [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)

In [ ]:
escalador = preprocessing.StandardScaler()
var_numericas_imputadas_escalado_standard = escalador.fit_transform(var_numericas_imputadas)

In [ ]:
escalador.mean_

In [ ]:
var_numericas_imputadas_escalado_standard.mean(axis=0)

In [ ]:
var_numericas_imputadas_escalado_standard.std(axis=0)

In [ ]:
var_numericas_imputadas_escalado_standard[0]

Para aquellos casos en los que los datos tengan muchos valores extremos, es posible que estandarizar usando la media y la desviacion estandar no funcione bien en el modelo. Para esos casos es mejor usar unos estimadores mas robustos (menos sensibles a outliers) y emplear un [RobustScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler) que funciona substrayendo la mediana y escalando mediante el rango intercuartil (IQR).

In [ ]:
escalador_robusto = preprocessing.RobustScaler()
var_numericas_imputadas_escalado_robusto = escalador_robusto.fit_transform(
                                                        var_numericas_imputadas)

In [ ]:
var_numericas_imputadas_escalado_robusto.mean(axis=0)

In [ ]:
var_numericas_imputadas_escalado_robusto.std(axis=0)

**Escalado a un rango especifico**

Hay casos en los que en vez de estandardizar queremos escalar los datos a un rango (generalmente [-1,1] o [0,1]). Para ello podemos usar [MinMaxScaler](scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler) que hace escalado minmax (obviamente) o [MaxAbscaler](scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbscaler) que simplemente divide cada valor de una variable por su valor máximo (y por tanto convierte el valor maximo a 1).

In [ ]:
var_numericas_imputadas.min()

In [ ]:
var_numericas_imputadas.max()

In [ ]:
escalador_minmax = preprocessing.MinMaxScaler()
var_numericas_imputadas_escalado_minmax = escalador_minmax.fit_transform(var_numericas_imputadas)

In [ ]:
var_numericas_imputadas_escalado_minmax.max()

In [ ]:
var_numericas_imputadas_escalado_minmax.min()

In [ ]:
escalador_maxabs = preprocessing.MaxAbsScaler()
var_numericas_imputadas_escalado_maxabs = escalador_maxabs.fit_transform(var_numericas_imputadas)

In [ ]:
var_numericas_imputadas_escalado_maxabs.max()

In [ ]:
var_numericas_imputadas_escalado_maxabs.min()

Hay casos en los que lo que se necesita es tener observaciones con norma unitaria (norma L2 o euclidiana). Para esos casos, podemos usar [Normalizer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html#sklearn.preprocessing.Normalizer)

In [ ]:
normalizador = preprocessing.Normalizer()
var_numericas_imputadas_normal = normalizador.fit_transform(var_numericas_imputadas)

Hay que tener en cuenta que el objetivo del Normalizer es normalizar casos, no variables (o sea que funciona por filas)

In [ ]:
var_numericas_imputadas_normal[1,:]

In [ ]:
np.linalg.norm(var_numericas_imputadas_normal[1,:])

<hr>

### Variables Categoricas

Los modelos están diseñados para trabajar con variables numéricas. Esto implica que para poder entrenar los modelos con variables categóricas tenemos que convertirlas a números. Este proceso se llama *codificación (encoding)*

In [ ]:
datos = pd.read_csv("datos_procesamiento.csv")
datos.head()

In [ ]:
var_categoricas = datos[['col_categorica', 'col_ordinal']]

In [ ]:
var_categoricas.head()

Hay muchas formas de codificar variables, la más sencilla es reemplazar los elementos de dichas variables por un número. Por ejemplo, si hacemos esto con la columna `col_ordinal`:

In [ ]:
label_codificador = preprocessing.LabelEncoder()
label_codificador.fit(datos.col_ordinal) 

In [ ]:
label_codificador.classes_

In [ ]:
label_codificador.transform(['muy bien', 'muy mal', 'muy bien', 'muy mal', 'bien'])

In [ ]:
label_codificador.inverse_transform([0, 0, 1, 2])

En el caso de variables ordinales esto tiene sentido ya que `muy_bien>bien>regular>mal>muy mal`. Sin embargo, esto indica a los modelos de scikit-learn por ejemplo que `mal + regular = bien`. Esto se puede usar en según que casos (hay modelos que no interpretan las variables numéricas así), o para codificar las variables objetivo.

Para variables categóricas (por ejemplo animales) no tiene sentido usar este tipo de encoding. 

In [ ]:
label_codificador_categorico = preprocessing.LabelEncoder()
label_codificador_categorico.fit_transform(datos.col_categorica)[:10]

In [ ]:
label_codificador_categorico.classes_

Digamos que no tiene sentido decir que la media de `elefante` y `perro` no es `gato`

Para estos casos una técnica que se puede usar se llama `one-hot encoding`. Lo que significa es que creamos n columnas binarias, con el valor 0 por defecto salvo la columna referente a la observación.

Para esto podemos usar 
[OneHotEncoder](scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [ ]:
oh_codificador = preprocessing.OneHotEncoder()

In [ ]:
oh_codificador.fit(datos.col_categorica)

Vemos que OneHotEncoder falla cuando se le pasan strings en vez de numeros. Por ello primero tenemos que convertir las variables categóricas a numéricas usando LabelEncoder.

In [ ]:
categorias_codificadas = label_codificador_categorico.transform(datos.col_categorica)

In [ ]:
categorias_codificadas

In [ ]:
categorias_codificadas.shape

In [ ]:
categorias_oh_codificadas = oh_codificador.fit_transform(categorias_codificadas.reshape(1000,1))
categorias_oh_codificadas

Vemos que por defecto `OneHotEncoder` no devuelve un numpy array, sino una matriz `sparse`. La traducción sería "matriz escasa", y es una manera de representar matrices con muchos ceros (como es el caso de OneHot encoding) para consumir poca memoria.

Podemos convertir dichas matrices a arrays facilmente:

In [ ]:
categorias_oh_codificadas.toarray()

Vemos ahora la comparación en memoria de una matriz sparse versus su correspondiente np.array usando la función `sys.getsizeof` que devuelve el uso de memoria un objeto de python en bytes.

In [ ]:
import sys
sys.getsizeof(categorias_oh_codificadas)

In [ ]:
sys.getsizeof(categorias_oh_codificadas.toarray())

Si queremos que el encoder devuelva arrays no tenemos más que pasarle el parametro `sparse=False`.

In [ ]:
oh_codificador = preprocessing.OneHotEncoder(sparse=False)

categorias_oh_codificadas = oh_codificador.fit_transform(categorias_codificadas.reshape(1000,1))
categorias_oh_codificadas

Pandas tiene la funcion auxiliar `get_dummies` que hace esto automáticamente de forma más fácil.

In [ ]:
pd.get_dummies(datos.col_categorica).head()

### Texto

In [ ]:
from sklearn import feature_extraction

In [ ]:
datos.col_texto.values[:10]

Para convertir texto en variables numéricas, podemos proceder de igual forma que con las variables categóricas, simplemente separando las palabras antes.

Para ello tenemos dos vectorizadores en scikit-learn, que convierten texto en vectores.


[CountVectorizer]() devuelve un vector con el valor 0 en todas las palabras que no existen en una frase y con el numero de ocurrencias de las palabras que si existen

Vamos a hacer un ejemplo para que se vea bien.

In [ ]:
ejemplo_frases = ['los coches rojos',
          'los aviones son rojos',
          'los coches y los aviones son rojos',
          'los camiones rojos'
                 ]


vectorizador_count = feature_extraction.text.CountVectorizer()
X = vectorizador_count.fit_transform(ejemplo_frases)
X

In [ ]:
vectorizador_count.get_feature_names()

In [ ]:
pd.DataFrame(X.toarray(), columns=vectorizador_count.get_feature_names())

El tomar simplemente el número de veces que aparece cada palabra tiene un problema, y es que da un mayor peso a aquellas palabras que aparecen muchas veces pero que no aportan ningun valor semántico (por ejemplo, `los`). Una manera más sofisticada de vectorizar un texto es en vez de usar el número de apariciones, usar TF-IDF. TF-IDF se traduce como Frecuencia de Texto - Frecuencia Inversa de Documento, y es una medida que asigna pesos a cada palabra en función de su frecuencia de aparición en todos los documents.

In [ ]:
vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
X = vectorizador_tfidf.fit_transform(ejemplo_frases)
pd.DataFrame(X.toarray(), columns=vectorizador_tfidf.get_feature_names())

In [ ]:
vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(datos.col_texto)
texto_vectorizado

In [ ]:
texto_vectorizado = texto_vectorizado.toarray()
texto_vectorizado

In [ ]:
label_codificador.classes_

**Poniendolo todo junto**

In [ ]:
col_numericas =  ['col_inexistente1', 'col2', 'col3', 'col_outliers', 'col_outliers2']
col_categorica = ['col_categorica']
col_texto = ['col_texto']


#Variables numéricas
imputador = SimpleImputer(missing_values=np.nan, copy=False, strategy="mean")
escalador = preprocessing.StandardScaler()
var_numericas_imputadas_escalado_standard = escalador.fit_transform(
                                                imputador.fit_transform(datos[col_numericas])
                                            )
df_numerico_procesado = pd.DataFrame(var_numericas_imputadas_escalado_standard,
                                                   columns=col_numericas)


# Variable categorica
label_codificador_categorico = preprocessing.LabelEncoder()
categorias_codificadas = label_codificador_categorico.fit_transform(datos[col_categorica])
oh_codificador = preprocessing.OneHotEncoder(sparse=False)
categorias_oh_codificadas = oh_codificador.fit_transform(categorias_codificadas.reshape(1000,1))

df_categorico_procesado = pd.DataFrame(categorias_oh_codificadas, 
                                       columns=label_codificador_categorico.classes_)


# Texto
vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(datos.col_texto)
df_texto_procesado =  pd.DataFrame(texto_vectorizado.toarray(), columns=vectorizador_tfidf.get_feature_names())


datos_procesados = pd.concat([
    df_numerico_procesado,
    df_categorico_procesado,
    df_texto_procesado 
], axis=1)

# variable ordinal
label_codificador_ordinal = preprocessing.LabelEncoder()
datos_procesados['col_ordinal'] = label_codificador_ordinal.fit_transform(datos.col_ordinal) 

In [ ]:
datos_procesados